In [ ]:
import panel as pn
pn.extension()

The `ChatEntry` is a widget for displaying chat messages with support for various content types.

This widget provides a structured view of chat messages, including features like:
- Displaying user avatars, which can be text, emoji, or images.
- Showing the user's name.
- Displaying the message timestamp in a customizable format.
- Associating reactions with messages and mapping them to icons.
- Rendering various content types including text, images, audio, video, and more.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **`value`** (object): The message contents. Can be a string, pane, widget, layout, etc.
* **`user`** (str): Name of the user who sent the message.
* **`avatar`** (str | BinaryIO): The avatar to use for the user. Can be a single character text, an emoji, or anything supported by `pn.pane.Image`. If not set, uses the first character of the name.
* **`reactions`** (List): Reactions associated with the message.
* **`reaction_icons`** (ChatReactionIcons | dict): A mapping of reactions to their reaction icons; if not provided defaults to `{"favorite": "heart"}`. Provides a visual representation of reactions.
* **`timestamp`** (datetime): Timestamp of the message. Defaults to the instantiation time.
* **`timestamp_format`** (str): The format in which the timestamp should be displayed.

##### Display

* **`show_avatar`** (bool): Whether to display the avatar of the user.
* **`show_user`** (bool): Whether to display the name of the user.
* **`show_timestamp`** (bool): Whether to display the timestamp of the message.
* **`name`** (str): The title or name of the chat entry widget, if any.

___

In [ ]:
pn.widgets.ChatEntry(value="Good to see you!", user="Happy User", avatar="😀")

Although `ChatEntry` is commonly used for `value`s of `str` type, it can support Panel panes, widgets, and layouts too!

In [ ]:
pn.widgets.ChatEntry(
    value=pn.pane.Markdown("## Happy you're here!"), user="Happy User", avatar="😀"
)

`ChatEntry` can be initialized without any input.

In [ ]:
chat_entry = pn.widgets.ChatEntry()
chat_entry

That way, the `value`, `user`, and/or `avatar` can be dynamically updated either by setting the param like this...

In [ ]:
chat_entry.value = pn.widgets.StaticText(value="Cheers!")

Or updating multiple values at once with the `.param.update` method!

In [ ]:
chat_entry.param.update(user="Jolly Guy", avatar="🎅")

The `timestamp` can be formatted using `timestamp_format`.

In [ ]:
pn.widgets.ChatEntry(timestamp_format="%b %d, %Y %I:%M %p")

Some active `reactions` can be associated with the message too.

In [ ]:
pn.widgets.ChatEntry(value="Love this!", reactions=["favorite"])

If you'd like to display other `reactions_icons`, pass a pair of `reaction` key to tabler `icon` name.

In [ ]:
pn.widgets.ChatEntry(
    value="Looks good!",
    reactions=["like"],
    reaction_icons={"like": "thumb-up", "dislike": "thumb-down"},
)

If you'd like a plain interface with only the `value` displayed, set `show_user`, `show_avatar`, and `show_timestamp` to `False` and provide an empty `dict` to `reaction_icons`.

In [ ]:
pn.widgets.ChatEbntry(value="Plain and simple", show_avatar=False, show_user=False, show_timestamp=False, reaction_icons={})

## Binding to reactions

Please provide an example. I cannot get it working.